<a href="https://colab.research.google.com/github/sath08/researchProject/blob/main/researchBiasCorrection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cftime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.3 MB/s eta 0:00:00


In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cython
import cftime
from scipy.io import netcdf
from tqdm import tqdm

In [ ]:
def fix_calendar(dataset):

  # Place all of the dates that should be removed into a boolean.
  removed_times = ~(((dataset.time.dt.month == 2) & (dataset.time.dt.day == 29)) | ((dataset.time.dt.month == 2) & (dataset.time.dt.day == 30)) | ((dataset.time.dt.month == 1) & (dataset.time.dt.day == 31)) | ((dataset.time.dt.month == 3) & (dataset.time.dt.day == 31)) | ((dataset.time.dt.month == 3) & (dataset.time.dt.day == 31)) | ((dataset.time.dt.month == 5) & (dataset.time.dt.day == 31)) | ((dataset.time.dt.month == 7) & (dataset.time.dt.day == 31)) | ((dataset.time.dt.month == 8) & (dataset.time.dt.day == 31)) | ((dataset.time.dt.month == 10) & (dataset.time.dt.day == 31)) | ((dataset.time.dt.month == 12) & (dataset.time.dt.day == 31)))

  # Return the dataset with the dates from removed_times excluded.
  return dataset.isel(time = removed_times)

In [ ]:
def perform_bias_correction():
    # Open and calibrate past_gcm dataset.
    past_gcm = xr.open_dataset(
        "/content/drive/MyDrive/Sathkrith_Segu_LumiereSummerProject/past_gcm/HADGEM_GCM_past.nc",
        decode_times=False,
    )
    past_gcm = xr.decode_cf(past_gcm)
    past_gcm = fix_calendar(past_gcm)

    # Open and calibrate future_gcm dataset.
    future_gcm = xr.open_dataset(
        "/content/drive/MyDrive/Sathkrith_Segu_LumiereSummerProject/future_gcm/ssp585/HADGEM_GCM_future.nc",
        decode_times=False,
    )
    future_gcm = xr.decode_cf(future_gcm)
    future_gcm = fix_calendar(future_gcm)

    # Open era5 dataset.
    era5 = xr.open_dataset(
        "/content/drive/MyDrive/Sathkrith_Segu_LumiereSummerProject/era5/ERA5_1980.nc"
    )
    for i in range(34):
        era5CurrYear = xr.open_dataset(
            "/content/drive/MyDrive/Sathkrith_Segu_LumiereSummerProject/era5/ERA5_"
            + str(1981 + i)
            + "_xarray.nc"
        )
        era5 = xr.concat([era5, era5CurrYear], dim="time")

    # Adjust past_gcm and future_gcm datasets to match era5.
    past_gcm_2015_val = future_gcm.sel(time=slice("2015-01-01", "2015-01-01"))
    past_gcm = past_gcm.sel(time=slice("1980-01-01", "2014-12-30"))
    future_gcm = xr.concat([past_gcm_2015_val, future_gcm], dim="time")

    # 1:1 copy, not a reference, of the future_gcm so it can be modified.
    future_gcm_BC_copy = future_gcm.copy()

    # Dictionary to store variables for each month.
    month_variables = {}

    for i in tqdm(range(1, 13)):

        # Variable name for each month.
        variable_name = f"month_{i}"

        # A month of data from the future to be bias corrected.
        future_gcm_BC_copy_month = future_gcm_BC_copy.sel(
            time=future_gcm_BC_copy["time.month"] == i
        )

        past_gcm_month = past_gcm.sel(time=past_gcm["time.month"] == i)["tas"]
        future_gcm_month = future_gcm.sel(time=future_gcm["time.month"] == i)["tas"]
        era5_gcm_month = era5.sel(time=era5["time.month"] == i)["t2m"]

        # latitude loops
        for lat in range(len(past_gcm.lat)):

            # Converts the latitude value at index phi into a float in the variable lat_phi
            lat = past_gcm.lat[lat].values

            # longitude loops
            for lon in range(len(gcm_P.lon)):

                # All the temperatures from month i at grid cell lat x lon sorted by temperature
                past_month_temp_grid_cell_sorted = np.sort(past_gcm_month[:, phi, L])
                future_month_temp_grid_cell_sorted = np.sort(future_gcm_month[:, phi, L])
                era5_month_temp_grid_cell_sorted = np.sort(era5_gcm_month[:, phi, L])

                # Creating a copy of future data
                future_gcm_month_cell = future_gcm_month[:, phi, L].values

                # Creates an array from 1/arr length to 1 which will be used for the percentile classification of temperatures
                # ECDF = Emperical Cumulative Distribution Function
                # First paramater (1) is the numerator for the index.
                # TODO understand the ECDF on Youtube Videos
                ecdf_past = np.arange(1, len(past_month_temp_grid_cell_sorted) + 1) / len(
                    past_month_temp_grid_cell_sorted
                )
                ecdf_future = np.arange(1, len(future_month_temp_grid_cell_sorted) + 1) / len(
                    future_month_temp_grid_cell_sorted
                )
                ecdf_era = np.arange(1, len(era5_month_temp_grid_cell_sorted) + 1) / len(
                    era5_month_temp_grid_cell_sorted
                )

                # Contains zeros in which we are going to save the Bias corrected values
                empty_future_gcm_month_cell = np.zeros(len(future_gcm_month_cell))

                # Loops through every future PV cell datapoint in the chronological order
                for i in range(len(future_gcm_month_cell)):
                    # Try catch
                    try:
                        # Does the ECDF (three arrows) mapping
                        empty_future_gcm_month_cell[i] = era5_month_temp_grid_cell_sorted[
                            np.where(past_month_temp_grid_cell_sorted <= future_month_temp_grid_cell_sorted[i])[0][-1]
                        ]  ### This is the main ECDF BC function.
                    except IndexError:
                        # If the condition is not fulfilled, it means that we found a value that is smaller than all of the data from the past which means that we need to use the first
                        # value in the sorted past which is the lowest temperature we have seen
                        empty_future_gcm_month_cell[i] = era5_month_temp_grid_cell_sorted[0]
                # Saves all of the datapoints for the latitude and longitude to the future_gcm_BC_copy_month
                future_gcm_BC_copy_month[PV].loc[dict(lat=lat, lon=lon)] = empty_future_gcm_month_cell

        # Adds 1 month of the BC data to the variables dictionary
        month_variables[variable_name] = future_gcm_BC_copy_month

    # merge the datasets:
    futr_PV_corr = xr.merge(
        [
            variables["month_1"],
            variables["month_2"],
            variables["month_3"],
            variables["month_4"],
            variables["month_5"],
            variables["month_6"],
            variables["month_7"],
            variables["month_8"],
            variables["month_9"],
            variables["month_10"],
            variables["month_11"],
            variables["month_12"],
        ]
    )

    # save the dataset:
    futr_PV_corr.to_netcdf(
        "/content/drive/MyDrive/Sathkrith_Segu_LumiereSummerProject/BC_dataset/HADGEM3_BC.nc"
    )
    return

In [ ]:
perform_bias_correction()